In [169]:
from z3 import *

import yaml
import pandas as pd

from dataclasses import dataclass
from pprint import pprint
from typing import Optional
from itertools import product

pd.set_option('display.max_rows', None)

In [170]:
MM_FILE = './assets/metamodels/doml_meta_v2.0.yaml'

IM_FILE = './assets/doml/2.0/nginx-openstack_v2.0.yaml'
# IM_FILE = './assets/doml/2.0/nginx-openstack_v2.0_wrong_vm_iface.yaml'

UB_ELEMS = 0

In [171]:
with open(MM_FILE, 'r') as mm_file:
    mm = yaml.safe_load(mm_file)

with open(IM_FILE, 'r') as im_file:
    im = yaml.safe_load(im_file)

We need to create first the elements by parsing the metamodel

In [172]:
classes = { f'{cat_k}_{elem_k}' : elem_v
    for cat_k, cat_v in mm.items()
    for elem_k, elem_v in cat_v.items() 
}

def merge_superclass(elem):
    e_k, e_v = elem
    sc_k = e_v.get('superclass')
    sc_v = classes.get(sc_k)
    if not sc_k:
        e_v['associations'] = {f'{e_k}::{k}': v for k,v in e_v.get('associations',{}).items()}
        e_v['attributes'] = {f'{e_k}::{k}': v for k,v in e_v.get('attributes',{}).items()}
        return e_v
    else:
        e_v['associations'] = {f'{e_k}::{k}': v for k,v in e_v.get('associations',{}).items()} | sc_v.get('associations',{})
        e_v['attributes'] = {f'{e_k}::{k}': v for k,v in e_v.get('attributes',{}).items()} | sc_v.get('attributes',{})
        return e_v

# all the elements with all the inherited attributes and associations
merged_classes = { 
    k : merge_superclass((k,v))
    for k,v in classes.items()
}

In [173]:

@dataclass
class Class:
    name: str
    attributes: dict[str, dict]
    associations: dict[str, dict]
    ref: Optional[SortRef] = None

@dataclass
class Elem:
    id: str
    name: Optional[str]
    attributes: dict[str, dict]
    associations: dict[str, dict]
    eClass: Optional[Class] = None
    ref: Optional[SortRef] = None
    unbound: bool = False

@dataclass
class AssocRel:
    from_elem: Class
    to_elem: Class
    inverse_of: Optional[str]
    ref: Optional[FuncDeclRef] = None

@dataclass
class AttrRel:
    multiplicity: str
    type: str
    ref: Optional[FuncDeclRef] = None

# Convert ELEMS
CLASSES = {
    class_k : Class(
        class_k, 
        class_v.get('attributes', {}), 
        class_v.get('associations', {})
    )
    for class_k, class_v in merged_classes.items()
}

ELEMS = {
    elem_k : Elem(
        elem_v['id'],
        elem_v['name'],
        elem_v['attrs'],
        elem_v['assocs'],
        CLASSES[elem_v['class']],
    )
    for elem_k, elem_v in im.items()
}

# This also helps catching errors in class/assoc names
ASSOCS = {
    f'{assoc_k}' : AssocRel(class_v, CLASSES[assoc_v['class']], assoc_v.get('inverse_of', None))
    for class_k, class_v in CLASSES.items()
    for assoc_k, assoc_v in class_v.associations.items()
}

# Careful: I decided to default multiplicity to 0..1
ATTRS = {
    f'{attr_k}' : AttrRel(attr_v.get('multiplicity', '0..1'), attr_v['type'])
    for class_k, class_v in CLASSES.items()
    for attr_k, attr_v in class_v.attributes.items()
}

Visualization of all the Elems and Assocs

In [174]:
classes_df = pd.DataFrame.from_dict([
    {
        'name': name
    }
    for name, value in CLASSES.items()
])

classes_df

,name
0,commons_DOMLElement
1,commons_Property
2,commons_IProperty
3,commons_SProperty
4,commons_FProperty
5,commons_BProperty
6,commons_Configuration
7,commons_Deployment
8,application_ApplicationLayer
9,application_ApplicationComponent


In [175]:
assoc_df = pd.DataFrame.from_dict([
    {
        'name': name,
        'from': value.from_elem.name,
        'to': value.to_elem.name,
        'inverse': value.inverse_of
    }
    for name, value in ASSOCS.items()
])

assoc_df[['from', 'name', 'to', 'inverse']]

,from,name,to,inverse
0,concrete_ComputingGroup,commons_DOMLElement::annotations,commons_Property,None
1,commons_BProperty,commons_Property::reference,commons_DOMLElement,None
2,commons_Configuration,commons_Configuration::deployments,commons_Deployment,None
3,commons_Deployment,commons_Deployment::component,application_ApplicationComponent,None
4,commons_Deployment,commons_Deployment::node,infrastructure_InfrastructureElement,None
5,application_ApplicationLayer,application_ApplicationLayer::components,application_ApplicationComponent,None
6,application_DBMS,application_SoftwareComponent::exposedInterfaces,application_SoftwareInterface,None
7,application_DBMS,application_SoftwareComponent::consumedInterfaces,application_SoftwareInterface,None
8,application_SaaSDBMS,application_SaaS::exposedInterfaces,application_SoftwareInterface,None
9,infrastructure_InfrastructureLayer,infrastructure_InfrastructureLayer::nodes,infrastructure_ComputingNode,None


## Z3 Sort Definitions
Here we define the Sorts of Elements and Associations as Enums (as they are finite), and the relationship between two elements as a Function `AssocRel :: ElemSort, AssocSort, ElemSort -> BoolSort`, which tells us if two items are in a relationship (returns true) or not.

In [176]:
# Init Z3 solver
s = Solver()

def Iff(a, b):
    return a == b

In [177]:
ub_elems = { f'elem_ub_{i}' : Elem(
    f'elem_ub_{i}',
    f'Unbound Element #{i}',
    {},
    {},
    unbound=True
    ) 
    for i in range(UB_ELEMS)
}

ELEMS |= ub_elems

In [178]:
class_sort, classes = EnumSort('Class', list(CLASSES.keys()))
# Add the Ref to each ELEM
for c in classes:
    CLASSES[str(c)].ref = c

elem_sort, elems = EnumSort('Elem', list(ELEMS.keys()))
for elem in elems:
    ELEMS[str(elem)].ref = elem

# Where assoc_sort is an EnumSort of all associations names...
assoc_sort, assocs = EnumSort('Assoc', list(ASSOCS.keys()))
# Add the Ref to each ASSOC
for assoc in assocs:
    ASSOCS[str(assoc)].ref = assoc

attr_sort, attrs = EnumSort('Attr', list(ATTRS.keys()))
# Add the Ref to each ATTR
for attr in attrs:
    ATTRS[str(attr)].ref = attr


In [179]:
# ElemClass(Elem) -> Class
elem_class_fn = Function('ElemClass', elem_sort, class_sort)

for _, elem in ELEMS.items():
    if not elem.unbound:
        user_friendly_name = f'({elem.name})' if elem.name else ''
        s.assert_and_track(elem_class_fn(elem.ref) == elem.eClass.ref, f'ElemClass {elem.id}{user_friendly_name} {elem.eClass.name}')

In [180]:
# AssocRel(Elem, Assoc, Elem) -> Bool
assoc_rel = Function('AssocRel', elem_sort, assoc_sort, elem_sort, BoolSort())

assoc_a = Const('assoc_a', assoc_sort)
for (_, e1), (_, e2) in product(ELEMS.items(), ELEMS.items()):
    if (not e1.unbound and not e2.unbound):
        stmt = ForAll(
            [assoc_a],
            Iff(
                assoc_rel(e1.ref, assoc_a, e2.ref),
                Or(
                    *(
                        assoc_a == ASSOCS[e1_assoc_k].ref
                        for e1_assoc_k, e1_assoc_v in e1.associations.items()
                        if e2.id in e1_assoc_v
                    )
                )
            )
        )
        s.assert_and_track(stmt, f'AssocRel {e1.id} {e2.id}')

In [181]:
# WORKING - 4 Jan 2023 17:30
# # AttrRel(Elem, Attr, Value) -> Bool
# attr_int_rel = Function('AttrIntRel', elem_sort, attr_sort, IntSort(), BoolSort())
# # ...do the others
# attr_a = Const('attr_a', attr_sort)
# attr_value_b = Const('attr_value_b', IntSort())
# for _, elem in ELEMS.items():
#     if elem.eClass:
#         eClassAttrs = elem.eClass.attributes
#         print(f'{elem.id} {elem.name}')
#         for k, v in eClassAttrs.items():
#             if v['type'] == 'Integer':
#                 print('\t', k)
#         stmt = ForAll(
#             [attr_a],
#             Iff(
#                 attr_int_rel(elem.ref, attr_a, attr_value_b),
#                 Or(
#                     *(
#                         attr_a == ATTRS[attr_name].ref
#                         for attr_name, attr_value in eClassAttrs.items()
#                         if attr_value['type'] == 'Integer'
#                     )
#                 )
#             )
#         )
#         s.assert_and_track(stmt, f'AttrIntRel {elem.id} Int')

# TEST - 4 Jan 2023 17:30 - WORKS
# AttrRel(Elem, Attr) -> Bool
# True if Elem has Attr relationship
# attr_int_exist_rel = Function('AttrIntExistRel', elem_sort, attr_sort, BoolSort())
# # ...do the others
# attr_a = Const('attr_a', attr_sort)
# for _, elem in ELEMS.items():
#     if elem.eClass:
#         eClassAttrs = elem.eClass.attributes
#         print(f'{elem.id} {elem.name}')
#         for k, v in eClassAttrs.items():
#             type = v['type']
#             print(f'\t{k} ({type})')
#         stmt = ForAll(
#             [attr_a],
#             Iff(
#                 attr_int_exist_rel(elem.ref, attr_a),
#                 Or(
#                     *(
#                         attr_a == ATTRS[attr_name].ref
#                         for attr_name, attr_value in eClassAttrs.items()
#                         if attr_value['type'] == 'Integer'
#                     )
#                 )
#             )
#         )
#         s.assert_and_track(stmt, f'AttrIntExistRel {elem.id}')

attr_int_exist_rel = Function('AttrIntExistRel', elem_sort, attr_sort, BoolSort())
elem_a = Const('elem_a', elem_sort)
attr_a = Const('attr_a', attr_sort)
for class_k, class_v in CLASSES.items():
    s.assert_and_track((
        ForAll(
            [elem_a],
            Implies(
                elem_class_fn(elem_a) == class_v.ref,
                ForAll(
                    [attr_a],
                    Iff(
                        attr_int_exist_rel(elem_a, attr_a),
                        Or(
                            *(
                                attr_a == ATTRS[i].ref
                                for i in class_v.attributes.keys()
                                if ATTRS[i].type == 'Integer'
                            )
                        )
                    )    
                )
            )
        )
    ), f'AttrIntExistRel {class_k}')

In [182]:
assert(s.check() == sat)

Since we're done with the setup, we need to be sure that the base model is satisfiable!

Right now we're not checking any requirement. We're just telling the solver which are the valid relationships between stuff.

In [183]:
vm, iface = Consts("vm iface", elem_sort)

req_vm_iface = ForAll(
    [vm],
    Implies(
        elem_class_fn(vm) == CLASSES["infrastructure_VirtualMachine"].ref,
        Exists(
            [iface],
            And(
                assoc_rel(vm, ASSOCS["infrastructure_ComputingNode::ifaces"].ref, iface)
            )
        )
    )
)
s.assert_and_track(req_vm_iface, "vm_iface")

In [184]:
vm = Const("vm", elem_sort)
CpuCount = Int("cpucount")

# req_vm_cpucount = ForAll(
#     [vm],
#     Implies(
#         elem_class_fn(vm) == CLASSES["infrastructure_VirtualMachine"].ref,
#         Exists(
#             [CpuCount],
#             And(
#                 attr_int_rel(vm, ATTRS["infrastructure_ComputingNode::cpu_count"].ref, CpuCount),
#                 CpuCount == 5
#             )
#         )
#     )
# )
# s.assert_and_track(req_vm_cpucount, "vm_cpucount")

req_vm_cpucount = ForAll(
    [vm],
    Implies(
        elem_class_fn(vm) == CLASSES["infrastructure_VirtualMachine"].ref,
        
        attr_int_exist_rel(vm, ATTRS["infrastructure_ComputingNode::cpu_count"].ref)
    )
)
s.assert_and_track(req_vm_cpucount, "vm_cpucount")

In `req_vm_cpucount` when using
```python
attr_int_exist_rel(vm, ATTRS["infrastructure_ComputingNode::cpu_count"].ref)
```
it works (produces *sat*) since `cpu_count` is indeed a property of VM.

If we use another attr. key instead, such as `infrastructure_KeyPair::keyfile`,
we get *unsat* as a result, which is consistent, since `keyfile` doesn't belong to the
properties of a VM.

In [185]:
isSat = s.check()
assert(isSat)
print(isSat)

sat


### Tests

In [186]:
# TEST: VirtualMachine must have only its own attribute relationship, and not others.
# VirtualMachine DOES NOT have Attribute infrastructure_Rule::cidr
# so we're expecting it to be unsatisfiable
s.push()
s.add(
    ForAll(
        [vm],
        Implies(
            elem_class_fn(vm) == CLASSES["infrastructure_VirtualMachine"].ref,
            attr_int_exist_rel(vm, ATTRS["infrastructure_Rule::cidr"].ref)
        )
    )
)
# If this fails, it means that you are assigning attribute relationship wrong!
assert s.check() == unsat
s.pop()

In [187]:
# Same test, for another component, but this time it MUST have its own attribute.
s.push()
s.add(
    ForAll(
        [vm],
        Implies(
            elem_class_fn(vm) == CLASSES["infrastructure_Rule"].ref,
            attr_int_exist_rel(vm, ATTRS["infrastructure_Rule::fromPort"].ref)
        )
    )
)
# If this fails, it means that you are assigning attribute relationship wrong!
assert s.check() == sat
s.pop()

In [188]:
s.check()
model = s.model()

In [189]:
model_df = pd.DataFrame([str(x).split(' ') for x in list(model)])

model_df.sort_values(by=0)

,0,1,2
0,AssocRel,elem_139682453244048,elem_139682453246288
528,AssocRel,elem_139682453246416,elem_139682465716176
529,AssocRel,elem_139682454976784,elem_139682453247120
530,AssocRel,elem_139682468739152,elem_139682453246288
532,AssocRel,elem_139682453252752,elem_139682454978448
534,AssocRel,elem_139682455132112,elem_139682454968848
535,AssocRel,elem_139682453248272,elem_139682453246288
536,AssocRel,elem_139682453248016,elem_139682453246288
537,AssocRel,elem_139682454978384,elem_139682465984400
538,AssocRel,elem_139682467223760,elem_139682467223760
